---
### **Import SK packages**

In [1]:
#r "nuget:Microsoft.SemanticKernel, 0.18.230725.3-preview"
//#r "nuget:Microsoft.SemanticKernel, 0.17.230718.1-preview"



Installed Packages Microsoft.SemanticKernel, 0.18.230725.3-preview

---

### **Load secrets into local variables**

In [2]:
#!import ../Settings/AppConfig.cs

//#!import /Settings/AppConfig.cs

AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Get AOAI values
string mydeployDavinci = apiConfig.deployDavinci;
string mydeployTurbo = apiConfig.deployTurbo;
string mydeployEmbed = apiConfig.deployEmbed; 
string myendpoint = apiConfig.endpoint;
string myAPIKey = apiConfig.APIKey;
string myQdrantEndpoint = apiConfig.QdrantEndpoint;

//NOTE: If you run the below writeline command, be sure to clear the output before pushing to repo!!
//Console.WriteLine(mydeployDavinci + " & " + mydeployEmbed + " & " + myQdrantEndpoint + " & " + myendpoint + " & " + myAPIKey);

---

### **Create the Semantic Kernel**

In [3]:
#!import ./Plugins/MathPlugin/MathPlugin.cs

using System.Text.Json;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning;
//using Plugins;

IKernel kernel = new KernelBuilder()
    //.WithAzureTextCompletionService(deploymentName: mydeployDavinci!, endpoint: myendpoint!, apiKey: myAPIKey!)    
    .WithAzureChatCompletionService(deploymentName: mydeployTurbo!, endpoint: myendpoint!, apiKey: myAPIKey!)    
    //.WithLogger(logger)
    .Build();

---

### **Import the Skill/Plug-in & Create a Planner object**

In [4]:
//AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Add the math plugin
var mathPlugin = kernel.ImportSkill(new MathPlugin(), "MathPlugin");

// Create a planner
var planner = new SequentialPlanner(kernel);

---

### **Generate the Sequential Plan**

In [5]:
// Create a planner
var planner = new SequentialPlanner(kernel);

//var ask = "I have $2130.23. How much would I have after it grew by 24% and after I spent $5 on a latte?";
var ask = "If my investment of 2130.23 dollars increased by 23%, how much would I have after I spent $5 on a latte?";
//var ask = "If I have 1000.00 dollars and spend 500.00 dollars, how much is left?";
var plan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Plan:\n");
Console.WriteLine(JsonSerializer.Serialize(plan, new JsonSerializerOptions { WriteIndented = true }));

Plan:

{
  "state": [
    {
      "Key": "INPUT",
      "Value": ""
    }
  ],
  "steps": [
    {
      "state": [
        {
          "Key": "INPUT",
          "Value": ""
        }
      ],
      "steps": [],
      "parameters": [
        {
          "Key": "number2",
          "Value": "1.23"
        },
        {
          "Key": "INPUT",
          "Value": "2130.23"
        }
      ],
      "outputs": [
        "INVESTMENT_AFTER_INCREASE"
      ],
      "next_step_index": 0,
      "name": "Multiply",
      "skill_name": "MathPlugin",
      "description": "Multiply two numbers. When increasing by a percentage, don\u0027t forget to add 1 to the percentage."
    },
    {
      "state": [
        {
          "Key": "INPUT",
          "Value": ""
        }
      ],
      "steps": [],
      "parameters": [
        {
          "Key": "number2",
          "Value": "5"
        },
        {
          "Key": "INPUT",
          "Value": "$INVESTMENT_AFTER_INCREASE"
        }
      ],
      "ou

---

### **Execute the Sequential Plan**


In [6]:
var result = await plan.InvokeAsync();

Console.WriteLine("Plan results:");
Console.WriteLine(result.Result.Trim());

Plan results:
2615.1829
